# **Google Data Analytics Certificate Capstone Project**

SQL, Google Sheets/Excel, Tableau

# INTRODUCTION

This is a case study for Bellabeat: a high-tech company that manufactures health-focused smart products, specifically designed for women. Their goal is to help their users be aware and understand their current habits to make healthier decisions. 

This case study is presented to the following stakeholders: 
1. Urška Sršen: Bellabeat’s cofounder and Chief Creative Officer
2. Sando Mur: Mathematician and Bellabeat’s cofounder
3. Bellabeat marketing analytics team

Products:
1. Leaf - Tracks activity, sleep and stress. Functional yet fashionable: can be worn as a necklace, bracelet or clip. 
2. Time - Watch that tracks activity, sleep and stress.
3. Spring - Water bottle that tracks water intake. 
4. Bellabeat App - Where users are able to track health data captured from their health devices/smart technology (Leaf, Time & Spring), including their menstrual cycle, and mindfullness habits. 


**Business Task:** Based on key trends in smart device usage, identify and provide insight on opportunities for growth on a particular Bellabeat product and provide recommendations for Bellabeat's marketing strategy. 

# DATA

To better understand trends in smart device usage, Sršen requested analysis on data from FitBit users from this Kaggle dataset: [Fitbit Fitness Tracker Data](https://www.kaggle.com/datasets/arashnic/fitbit). This CC0 Public Domain dataset is made available by [Möbious](https://www.kaggle.com/arashnic). The dataset includes personal tracker data from 30 FitBit users. It includes data on the users' daily activity, steps, calories, heart rate, and sleep. All data is stored in CSV files which are easily importable to various analysis tools (Excel, R, SQL, etc.). Although Sršen encouraged the use of other data to perform the analysis, only this dataset was used for this case study. The tools used in this analysis are Google Sheets/Excel and Big Query (SQL). 

*Exploring, Verifying and Cleaning the Data*

Before importing the files into SQL, all columns that had a date and time (e.g. ActivityHour) were formatted to DATE TIME in Google Sheets. Otherwise, SQL would not be able to auto-detect the schema. The following CSV files were imported:

1. dailyActivity_merged AS *daily_activity_merged* 
2. dailyIntensities_merged AS *daily_intensities*
3. sleepDay_merged AS *daily_sleep*
4. hourlyCalories_merged AS *hourly_calories*
5. hourlySteps_merged AS *hourly_steps*
6. weightLogInfo_merged AS *weight_log*

In [ ]:
## Finding out how many distinct IDs there are in each table to verify how much data we have in each table and explore which function of their FitBit are users religiously using and/or tracking. The same code is used for daily_sleep, hourlyCalories, hourlySteps, and weight_log, with FROM modified to the appropriate table name. The daily_intensities table is not included since it is extracted from the daily_activity_merged table.  ##

SELECT 
DISTINCT Id,
COUNT (Id) AS number_of_logs
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY Id
ORDER BY Id

Number of Unique IDs per table:

<iframe width="383" height="166" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vTpdOfWvuWZWZdYwG7qRvibQ2rc2BGxGKT374el3fWbj_VhRnBbp78iuoVA7fK16QANc0B94lZFilTO/pubchart?oid=989483410&amp;format=interactive"></iframe>

A more detailed view can be found [here.](https://docs.google.com/spreadsheets/d/1VGFnbuwsAkGL_GEO6qrmTIwJAPBf-SxT9fC5S4dqWcM/edit?usp=sharing) This includes the list of distinct IDs per table, and the number of entries (or 'logs') per ID.

**Insights**: 
* The dataset actually has 33 participants, not 30 as previously described. Also, it encompasses a 31-day period, although not all users have tracked data for all 31 days. Begins 2016-04-12 and ends at 2016-04-12.
* The weight_log dataset will not be used for analysis because there is not enough data to draw meaningful insights. 
* There is no information about the demographics of the sample collected which will limit our analysis. 

# ANALYSIS

**1. How consistent are users wearing their Fitbit to track their daily activity?**

Users are categorized according to how often they used their FitBit throughout the 31-day period. 

* 25-31 days = Very Consistent User
* 15-24 = Consistent User
* 0-14 = Inconsistent User

In [ ]:
## How consistent are our users using their FitBit to track their daily activity? Users are broken down into three categories depending on how many days they used their FitBit to track their activity.##

SELECT  
  Id,
  Count(Id) AS num_of_logs,
CASE
WHEN Count(Id) BETWEEN 25 AND 31 THEN 'Very Consistent User'
WHEN Count(Id) BETWEEN 15 AND 24 THEN 'Consistent User'
WHEN Count(Id) BETWEEN 0 AND 14 THEN 'Inconsistent User'
 END UserActivity
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY 
  Id


<iframe width="558" height="345" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vRNobizTvXWKlOdhSJxkl1oaOfVX6oZQBorSSHLgrppYnvricNBYshJ9uJJbUzcFJN5pWyHgLufQK48/pubchart?oid=158383963&amp;format=interactive"></iframe>

* The data shows that majority of the users are consistent at wearing their FitBits to track their daily activity, with only 1 user inconsistent. 
* Bellabeat should set up reminders to encourage the use of their device, and create a reward system for being consistent user. 

The result of the code can be viewed [here.](https://docs.google.com/spreadsheets/d/1Yg-s-qz6qRIrRkSFDLgO78CdBPzFN-geKhr7K52Jc7w/edit?usp=sharing)

**2. How active are the users?**

To provide insight on the trends in the users' activity, we will primarily use the daily_activity_merged table. Data on daily steps, calories, and active & sedentary minutes will be analyzed and discussed. 

**2.1 STEPS**

First, I looked at the summary statistics of the users' step count. 

In [ ]:
## Summary values on users' daily steps including the num_of_logs to see if data may be skewed ##

SELECT 
Id,
max(TotalSteps) as max_TotalSteps,
min(TotalSteps) as min_TotalSteps,
avg(TotalSteps) as ave_TotalSteps,
COUNT(Id) as num_of_logs
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY Id

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQMAI6ufnJejp3lZWorQVD7-wx3BlejLvhEBrCUrKObuTna0S7_r-8LjEQHLyajeMrgidklquWdf_Ct/pubchart?oid=2069508337&amp;format=interactive"></iframe>

Result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1Bink96_iWwyyzPussjdQlXqXKnjtOAihnrNRsJ6N6Sw/edit?usp=sharing)

Majority of the users are hitting between 5000 to 10000 steps daily, but I wanted to see more clearly if most of them are  hitting the recommended 10,000 steps per day goal.

In [ ]:
## Avg total steps per user to see if they meet the recommended 10k step goal ##

SELECT  
  Id,
  avg(TotalSteps) as avg_TotalSteps,
CASE
WHEN avg(TotalSteps) >= 10000 THEN 'Meets 10k step-goal'
WHEN avg(TotalSteps) < 10000 THEN 'Does not meet 10k step-goal'
END Meet10kStepGoal
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY 
  Id

<iframe width="658" height="368" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQyE60uh6OtyI-Z9PzC-Sdba8fVT-04NiOMFCysJv7xWdKJOK5GxI9ASTRbo0X6_AO1IrN0_gbNYLRP/pubchart?oid=2005458234&amp;format=interactive"></iframe>

Results of the code can be viewed [here.](https://docs.google.com/spreadsheets/d/11SLQ7uUiE6hOiU-NJL5sWPElJ-b1hOi9qHT4Wdw0nI8/edit?usp=sharing)

With 26 users not meeting the recommended 10k step goal, we can encourage setting daily step goals and set up reminders to hit goals throughout the day and recognize milestones when higher step numbers are reached. 

**2.2 Active and Sedentary Minutes**

Next, I wanted to see how many minutes of the users' day are usually spent on physical and sedentary activity. The CDC recommends at least 150 minutes of physical activity per week, which means around 30 minutes per day or 600 minutes per month.


In [ ]:
## The code below totals the avg of the users' VeryActiveMinutes and FairlyActiveMinutes and then identifies which of the users meet the CDC recommendation on a daily average basis.##

SELECT 
Id,
avg(VeryActiveMinutes) + avg(FairlyActiveMinutes) AS avg_ActiveMinutesPerDay,
CASE
WHEN (avg(VeryActiveMinutes) + avg(FairlyActiveMinutes)) >= 30 THEN 'Yes'
WHEN (avg(VeryActiveMinutes) + avg(FairlyActiveMinutes)) < 30 THEN 'No'
END MeetsCDCRecommendation
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY Id

<iframe width="601" height="372" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vTeRtNDnKG-nWHmsaEOThgMo2kIGl6XOQDNIHQpU3w2o7cIAD0-NKZx4b3-LKJvmTquxBJBY-UpKSqn/pubchart?oid=1788725973&amp;format=interactive"></iframe>

Results of the code can be viewed [here.](https://docs.google.com/spreadsheets/d/17uimjgFiU4oV7McAfGpyMwvpv3QHOHB1pBG5POWetcE/edit?usp=sharing)

Only 17/31 users meet the recommended 30 Active minutes per day with their average active minutes. To see if these numbers would change if we took the total Active minutes spent throughout the entire month, I ran the code below.  

In [ ]:
## How many of the users met the recommended Active minutes this month?##

SELECT 
Id,
sum(VeryActiveMinutes + FairlyActiveMinutes) AS sum_ActiveMinutesEntireMonth,
CASE
WHEN sum(VeryActiveMinutes + FairlyActiveMinutes) >= 600 THEN 'Yes'
WHEN sum(VeryActiveMinutes + FairlyActiveMinutes) <600 THEN 'No'
END MeetsCDCRecommendation
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY Id

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vToFzzy_8oZVtRJKbn0A1itZdKTZm1ym1yHh8RlQyesk5vnEkJyD6UKWyNENpKYM4b9_9sRLWG5Kz8-/pubchart?oid=229601266&amp;format=interactive"></iframe>

Results of the code can be viewed [here.](https://docs.google.com/spreadsheets/d/1o-lukxvJN4__djl_OfhMwqTGK4cIRvlB_aGrRJAppfk/edit?usp=sharing)

Even if some users did not meet the recommended Active minutes averaged daily, 20/31 users did meet the recommended Active minutes this month which is at least 600 minutes. This shows that not all days are the same, and there are days when users have more time to be active. 

**2.3 Steps + Activity: How do these vary throughout the day and throughout the week?**

First, let's see how steps varied depending on the time of day, and then day of the week. 

In [ ]:
## Query below extracted date and time from ActivityHour column, and added day of the week, where 1 = Sunday. I used the table below to find out the average steps users took depending on the time of the day. ##

SELECT
ActivityHour,
EXTRACT(DATE FROM ActivityHour) AS dateonly,
EXTRACT(TIME FROM ActivityHour) AS timeonly,
EXTRACT (DAYOFWEEK FROM ActivityHour) AS dayoftheweek,
avg(StepTotal) AS avgTotalSteps
FROM `bellabeat-case-study-353900.Fitabase_Data.hourlySteps`
GROUP BY ActivityHour

<iframe width="653" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vRLaG4RCbsaSEQ869OQuIs6XPrO_FCc3gwwkJkROI39EDgWy82Q8Blxp9LKtk4yD9Ltw_IC8b4kisK8/pubchart?oid=1375086423&amp;format=interactive"></iframe>

Results of the code can be found [here.](https://docs.google.com/spreadsheets/d/1OGfScgMRGb_odrWG4UVbJoNgKm7rpPLRN_yLL9a7l8Y/edit?usp=sharing) The 'weekday' column was added manually through Google Sheets and the average per hour was done through a pivot table. Still trying to master doing all these calculations through SQL, but this should do for now.

Generally, there are two parts of the day where steps typically peak. Those two parts are from 12:00 - 14:00 and 17:00 - 19:00. The 'drops' in the graph seem to appear after meal times, which is after 14:00 and after 19:00. It may be a good idea to suggest users to take a walk after dinner to increase steps and activity before winding down for the day. 

In [ ]:
## I used the table below to find out the average steps users took depending on the day of the week. ##

SELECT
*,
EXTRACT(DATE FROM ActivityHour) AS dateonly,
EXTRACT (DAYOFWEEK FROM ActivityHour) AS dayoftheweek,
FROM `bellabeat-case-study-353900.Fitabase_Data.hourlySteps`


<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vTMhZhrGotK9crN2ZyeVr7OrryOdibXX0XlLbFntli9ypBxQTr5Uqcn5s-J2T5fh12Zy3HXgxE7Pm51/pubchart?oid=977048867&amp;format=interactive"></iframe>

Result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1XArUGZ8zMn6MVx6-skU6f3jUCpVIsTg_UEJWf-uv1Qc/edit?usp=sharing) Weekday column and pivot table were manually done on Google Sheets. 

There's not much difference in the number of steps taken depending on the day of the week, with the highest being on Saturdays at 8964 and the lowest on Mondays at 7659. One thing to note is that none of these averages reach the recommended 10,000 step goal. 

Next, I checked to see how Activity varied thoroughout the week for both Active Minutes and Sedentary Minutes. 

In [ ]:
## Sedentary minutes depending on day of the week ##

SELECT
ActivityDay,
EXTRACT (DAYOFWEEK FROM ActivityDay) AS dayoftheweek,
SedentaryMinutes
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_intensities`
GROUP BY dayoftheweek


<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vT2HoA_Jq8m-8yKOUKykj3mX4FWQ9bZvsvJi-RTboEFIZxj23DTZKaAbTP8jke8ZUrDFuFW-FbilGDW/pubchart?oid=1660458937&amp;format=interactive"></iframe>

Results of the code can be found [here.](https://docs.google.com/spreadsheets/d/1RBDK6EQ3HnPkhv95Q7HOHlhn0hR4brmJ_SDO4YfdSjY/edit?usp=sharing)

Starting off with looking at Sedentary Minutes, there is not much variation on the numbers depending on the day of the week although I thought weekends would have some more time spent relaxing. The users generally spend around 16-17 hours of their day sedentary. A portion of that could be sleep.

In [ ]:
## Total Active Minutes depending on day of the week. ##

SELECT
Id,
ActivityDay,
EXTRACT (DAYOFWEEK FROM ActivityDay) AS dayoftheweek,
LightlyActiveMinutes, FairlyActiveMinutes, VeryActiveMinutes, 
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_intensities`


<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQ-cOV_B91dfkjmRUocL0qjClR1jbDcp6HOJHWf9wqxvB4Xo_R53Ulg2UjUVD6gWkXaYHFD-SXzPghs/pubchart?oid=359342709&amp;format=interactive"></iframe>

Results of the code can be found [here.](https://docs.google.com/spreadsheets/d/1pDqkM3GcRvvpX02Tvwgd4UtuGcW9norJn-ePxX3DuvI/edit?usp=sharing) 

Again, there is also not much variation in the Active Minutes depending on the day of the week, with Tuesday, Friday and Saturday being the top 3. The users typically spend 3.5-4 hours of their day for active (Light Activity - Very Active Activity). 

**2.4 Calories**

It is hard to tell if this data point means calories burned or consumed but for the sake of the case study it will be assumed as calories burned because it is not shown as manually inputted data by the user.

In [ ]:
## Running summary statistics for Calories with the number of logs to account for any data skewed and guidance in analysis. ##

SELECT 
Id,
max(Calories) as max_Calories,
min(Calories) as min_Calories,
avg(Calories) as avg_Calories,
Count(Id) as num_of_logs
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`
GROUP BY Id 


<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vRS4avUyDA3feabIHXztvil1NH68wNzCbV2SMkQeQSOu3yYiOpZsPugUQcoxu5NVQAo8_8yF0FHQ-82/pubchart?oid=466247123&amp;format=interactive"></iframe>

Result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1gckzZb3LKv6eryTRu678zE6Ptr6VVxsin1xNCUak6cA/edit?usp=sharing)

The data shows that 20/33 users burn at least 2000 calories per day, with 12 of those users burning at least 2500 calories per day. I cannot make any conclusions as to whether this is good or bad in terms of health since Calories depend on so much on other factors that we do not have data on (e.g. gender, muscle mass, goals). 

**2.5 Sleep**

First, I looked at how consistent the users used their FitBit to track their sleep. 

In [ ]:
## Query below shows how often users wore their fitbit device to capture sleep data. ##

SELECT
Id,
COUNT(Id) as num_of_logs,
CASE
WHEN Count(Id) BETWEEN 25 AND 32 THEN 'Very Consistent User'
WHEN Count(Id) BETWEEN 15 AND 24 THEN 'Consistent User'
WHEN Count(Id) BETWEEN 0 AND 14 THEN 'Inconsistent User'
 END UserActivity
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`
GROUP BY Id
ORDER BY num_of_logs DESC

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQYDj4EK2M_GKaykJt3bKk8sATXN7LmFNN1Ht7obWzC1hcXFsCPbSkey0URvteKjZ0896RKhnNlOKOl/pubchart?oid=1235487061&amp;format=interactive"></iframe>

Results of the code can be viewed [here.](https://docs.google.com/spreadsheets/d/1bAoWtbRsHBX3AqCS5XH1CweE9KjnuXHJdPUlvNcagRU/edit?usp=sharing)

Although 24 users logged their sleep in the 31-day period, not all of the 24 users have 31 days worth of sleep data. I kept this in mind while doing my analysis. 

In [ ]:
## Query includes average time spent in bed and average minutes asleep ##

SELECT
Id,
COUNT(Id) as num_of_logs,
AVG(TotalTimeInBed) AS avg_TotalTimeInBed,
AVG(TotalMinutesAsleep) AS avg_TotalMinutesAsleep
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`
GROUP BY Id
ORDER BY num_of_logs DESC

<iframe width="600" height="110" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQDSF434ZN6sWVCj0xl224gCUpnN4MSlebqkGKYBOyOcYQ7mBvQB_gYlUgB4DEvwHzvvqWuU6HBhUtw/pubchart?oid=1182731559&amp;format=interactive"></iframe>

Results of the code can be found [here](https://docs.google.com/spreadsheets/d/1RCkJFG3I_-D0elJMA1YazOryUj43ZtUSZg9seSPVYyk/edit?usp=sharing). 

Because the inconsistent users' sleep data seemed to skew the results, I included two different averages for analysis. Generally the users have an average of 6-7 hours of sleep with 30-40 minutes of 'downtime' in bed before falling asleep. 

Because the sleep data was in itself unreliable, I decided not to proceed with analyzing the trends of sleep depending on the day of the week. 

**3. Relationships between variables**

How does sleep, steps, activity and calories affect one another? 

**3.1 Does more Steps mean more Calories burned?**

In [ ]:
## Does more steps = more calories? Trend line shows positive relationship between steps and calories. ##

SELECT  
TotalSteps,
Calories
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`


<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQzezHJSCSISfo4EA_FTbmar7QPWYWLwBMBxf5p7wzV0Y_H5flLSM58hECRAIDf0XSYVh7r5HLjebtb/pubchart?oid=323020600&amp;format=interactive"></iframe>

The trendline shows that there's a positive relationship between Steps and Calories. This means that when Steps increase, Calories burned will also increase.


The result of the code can found [here.](https://docs.google.com/spreadsheets/d/1Q5tRBOx5Fxw6hy2eX2-W7F-ZS7s1K6ZkptK79lO58a0/edit?usp=sharing)

**3.2 Does more Active Minutes mean more Calories burned?**

In [ ]:
## Relationship between active minutes and calories ##

SELECT  
VeryActiveMinutes, FairlyActiveMinutes, LightlyActiveMinutes, Calories
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vQBhZT16BopdcNer_MhtTFmhjivMA7XPMdhrX9Vbi9bvHUbwmw2qygOAgogo2ntkW71oNh-CTG5pUBU/pubchart?oid=1281692653&amp;format=interactive"></iframe>

The trendline shows that there's a positive relationship between the amount of Active Minutes and Calories. This means that when Active minutes increase, Calories burned will also increase.

The result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1KykR_geBJG4qMZVsp79RDeJvffjDzjn1F3QrXdm1Jms/edit?usp=sharing)

**3.3 How does Sedentary Minutes affect Calories burned?**

In [ ]:
## Relationship between sedentary minutes and calories ##

SELECT  
SedentaryMinutes, Calories
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSVoiT_jsbI7j_IcZG921wP20mP2mFLkqHch3XJ7ewIsNZpOFMGsurxhtMB9aCvF9yWAI1ShhgKNGil/pubchart?oid=1919159923&amp;format=interactive"></iframe>

The trendline shows that there's a negative relationship between the amount of Sedentary Minutes and Calories. This means that if Sedentary Minutes increase, Calories burned will decrease. 

The result of the code can be found [here. ](https://docs.google.com/spreadsheets/d/1pKHFzA7Q0rg_-n0WZAZobpB7LA-U01b8ySv7SeEJwgw/edit?usp=sharing)

**3.4 How does Calories burned affect Sleep?**

In [ ]:
## Used Inner Join to combine Sleep and Daily Activity Data to see how amount of calories spent affects sleep. ##

SELECT
`bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.id, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.Calories, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.ActivityDate, `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.SleepDay, `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.TotalMinutesAsleep
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`
INNER JOIN `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged` ON `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.id=`bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.Id AND `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.SleepDay=`bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.ActivityDate

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vTPSPmznGfRU3QOh0GmVu-sTYiI5Y14Eu1bg3oOmq4buIO_CsLeL7z1DfkcLhg7S6Ktu8hQFfTuoDoG/pubchart?oid=705623216&amp;format=interactive"></iframe>

It looks like this trendline is almost flat, indicating that there's no relationship between Calories and Sleep. In other words, no matter what the value of Calories is, it will not affect Sleep. The slope of the trendline is actually -0.0000738, which is very close to zero. 

The result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1t2zX2-jjXDQdtGPK8etOhIDOFJKVOJG9KzHYXtNdW1I/edit?usp=sharing)

**3.5 How does Steps affect Sleep?**

In [ ]:
## Compared total steps to total sleep ##

SELECT
`bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.id, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.TotalSteps, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.ActivityDate, `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.SleepDay, `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.TotalMinutesAsleep
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`
INNER JOIN `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged` ON `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.id=`bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.Id AND `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.SleepDay=`bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.ActivityDate

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSlR8OEdrVs24q3sD5T73ZRbIE_UVXJS6LpXauPXmZLKM58ShUNm-cY3-wnfW2GnOqE9gTFC4qzn4lR/pubchart?oid=750520988&amp;format=interactive"></iframe>

Surprisingly, there's a negative relationship between Steps & Sleep. This means that when Steps increase, number of hours sleep decrease. 

The result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1FBwqUulQ87e294bb4DP9wlLkmHX4Bk9uGtAioU8hfps/edit?usp=sharing)

**3.6 How does Active and Sedentary Activity affect Sleep?**

In [ ]:
## How does activity affect sleep? ##

SELECT
`bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.id, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.VeryActiveMinutes, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.FairlyActiveMinutes, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.LightlyActiveMinutes, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.SedentaryMinutes, `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.ActivityDate, `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.SleepDay
FROM `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`
INNER JOIN `bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged` ON `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.id=`bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.Id AND `bellabeat-case-study-353900.Fitabase_Data.daily_sleep`.SleepDay=`bellabeat-case-study-353900.Fitabase_Data.daily_activity_merged`.ActivityDate

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSm11u79IbxNuoPcF6O2bhHydb5n-_825PMPCn59Hxqev6htkBgxnDTuU_NFcP4Rm0fDv5M_ubFzv9f/pubchart?oid=1883377337&amp;format=interactive"></iframe>

Similar with Calories and Sleep, the trendline looks flat for Active Hours and Sleep, indicating that there's no relationship between the two variables. The slope of the trendline is -0.0815. 

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSm11u79IbxNuoPcF6O2bhHydb5n-_825PMPCn59Hxqev6htkBgxnDTuU_NFcP4Rm0fDv5M_ubFzv9f/pubchart?oid=1261479805&amp;format=interactive"></iframe>

On the other hand, the graph shows that there is a negative relationship between Sedentary Hours and Sleep. This means that the more time spent Sedentary, the less total sleep a user has. However, I realized that the Sedentary Hours could include some time slept. To remove the time spent sleeping in the Sedentary Hours, I decided to subtract 7 hours from all the values since that is the average number of hours slept. This will give an estimated number of hours doing sedentary activity, minus sleep. 

<iframe width="600" height="371" seamless frameborder="0" scrolling="no" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSm11u79IbxNuoPcF6O2bhHydb5n-_825PMPCn59Hxqev6htkBgxnDTuU_NFcP4Rm0fDv5M_ubFzv9f/pubchart?oid=1472259523&amp;format=interactive"></iframe>

Even with the adjusted Sedentary Hours, the result still shows a negative relationship between Sedentary Hours and Sleep. 

The result of the code can be found [here.](https://docs.google.com/spreadsheets/d/1q7WFeFss-5x6TaM7u8UwCtg7yx4DZelKT89Ja_8JG4A/edit?usp=sharing)

# TABLEAU VIZ #

I created a dashboard that summarizes the results of the analysis. The dashboard can be found [here!](https://public.tableau.com/views/BellabeatCaseStudyLearningfromFitBitUsers/DashboardTrendsinDailySteps?:language=en-US&:display_count=n&:origin=viz_share_link)



# SUMMARY & RECOMMENDATIONS

1. It is important to note that this dataset only includes data from 33 participants, with no demographic information. Because of this, we cannot conclude that the results of this analysis can be applied to Bellabeat's target population which is women.

2. To gain a better insight on their users, Bellabeat users should be encouraged to include demographic and health information when setting up their Bellabeat app and device.They should also be highly encouraged to set up their health goals so that the Bellabeat application can be tailored to their needs and likings. These goals and health information (e.g. weight) should be periodically updated so that they can recognize (and celebrate) their achievements and they can create new goals. 

3. Fitbit users most popular features are tracking the following: Steps, Activity, and Sleep. For this reason, I am providing recommendations based on my analysis on these health data. These recommendations are tailored to Bellabeat's product 'Ivy' and the Bellabeat App since both these products track this information.

* Steps - Because majority of the Fitbit users did not hit the recommended 10,000-step goal, Bellabeat should ensure that users are aware of how many steps they are taking daily and be encouraged to reach their own step goals. Bellabeat's users should be reminded when their step counts are looking low for the day so that steps can be encouraged. Users can also be rewarded (notified) when increments of their step goal have been reached, which can motivate them to reach their daily goal. 
* Activity - Although majority of the Fitbit users were able to hit the recommended Active minutes, Bellabeat can do better by giving options (through the Bellabeat app) to their users on how they can increase their activity throughout day, tailored to their bodies needs. Suggestions can include how much is the recommended time to do the activity, and the calories burned.
* Sleep - Even if the data from the Fitbit users was lacking for the most part, it showed that users are interested and somewhat consistent in tracking their sleep data. Fitbit users have an average of 6-7 hours of sleep a night. Bellabeat should encourage their users to also get at least 7 hours of sleep every day. They can do this by setting a bed time and a wake up time so that the time to sleep can be calculated for the user. They can include downtime meditation in their Bellabeat app to help their users fall asleep in a calm and relaxed manner. To gain more insight on their user's sleep, the Bellabeat app can ask for feedback to see how the user felt rested through their sleep. 

4. To increase their users, Bellabeat can focus on marketing their Bellabeat app and the features that it can provide to their users:

* Personal and tailored to your needs.
* Weekly and monthly reporting to track progress and opportunites for improvement in health habits.
* Adding projection data for their users (i.e. If they are consistent with their good habits, X results can happen).
* New activities to try to improve health data (e.g. Ideas to increase steps/activity for the day). Partner with other businesses (e.g. Yoga studios) to give users perks or discounts to encourage users to try new activities or to simply get moving. 
